In [1]:
'''This script demonstrates how to build a variational autoencoder with Keras.

Reference: "Auto-Encoding Variational Bayes" https://arxiv.org/abs/1312.6114
'''
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from keras.layers import Input, Dense, Lambda, BatchNormalization
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist

batch_size = 110
original_dim = 978
# batch_size = 100
# original_dim = 784

latent_dim = 2
intermediate_dim = 256
epochs = 50
epsilon_std = 1.0

x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)


def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)


def vae_loss(x, x_decoded_mean):
    xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss


Using Theano backend.


In [2]:
vae = Model(x, x_decoded_mean)
vae.compile(optimizer='rmsprop', loss=vae_loss)

# train the VAE on MNIST digits
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# vae.fit(x_train, x_train,
#         shuffle=True,
#         epochs=epochs,
#         batch_size=batch_size,
#         validation_data=(x_test, x_test))

# # build a model to project inputs on the latent space
# encoder = Model(x, z_mean)

# # display a 2D plot of the digit classes in the latent space
# x_test_encoded = encoder.predict(x_test, batch_size=batch_size)
# plt.figure(figsize=(6, 6))
# plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test)
# plt.colorbar()
# plt.show()


In [3]:
## Load data from h5 file
import h5py
f = h5py.File('../data/CD_matrix_105939xlm978.h5', 'r')
mat = f['matrix']
sig_ids = f['meta']['sig_ids']
print mat.shape, type(mat), len(sig_ids)
print mat.dtype

residual = mat.shape[0] % batch_size

n_train = 950 * batch_size
n_test = mat.shape[0] - residual - n_train 

(105939, 978) <class 'h5py._hl.dataset.Dataset'> 105939
float32


In [4]:
from sklearn.preprocessing import MinMaxScaler

In [5]:
## subset into train and test

scl = MinMaxScaler()

f2 = h5py.File('../data/CD_matrix_%sxlm978_train-test.h5' % mat.shape[0], 'w')
dset_train = f2.create_dataset('train', shape=(n_train, 978))
dset_train.write_direct(scl.fit_transform(mat[:n_train, :]))

dset_test = f2.create_dataset('test', shape=(n_test, 978))
dset_test.write_direct(scl.fit_transform(mat[n_train:(mat.shape[0]-residual), :]))
f2.close()

In [6]:
f2 = h5py.File('../data/CD_matrix_105939xlm978_train-test.h5','r')
mat_train = f2['train']
mat_test = f2['test']

print mat_train.shape, mat_test.shape
print mat_test[:,:].min(), mat_test[:,:].max()
print mat_train[:,:].min(), mat_train[:,:].max()


(104500, 978) (1430, 978)
0.0 1.0
0.0 1.0


In [7]:
vae.fit(mat_train, mat_train,
        shuffle="batch",
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(mat_test, mat_test)
       )

Train on 104500 samples, validate on 1430 samples
Epoch 1/50
104500/104500 [==============================] - 18s - loss: 652.5374 - val_loss: 668.8386

KeyboardInterrupt: 

In [12]:
print 105939 % 110
print 105939 / 110

9
963


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

print x_train.shape, x_test.shape
print x_train.min(), x_train.max()

(60000, 784) (10000, 784)
0.0 1.0
